感知机设定权重的工作，即确定合适的、能符合预期的输入与输出的权重，是通过人工进行的

神经网络的出现就是为了解决该问题，神经网络的一个重要性质是它可以自动地从数据中学习到合适的权重参数。

### 一. 多维数组

In [1]:
import numpy as np

In [3]:
a = np.array([[1,2], [3,4]])
b = np.array([[2,2], [2,2]])
# 逐位相乘
c = a * b
c

array([[2, 4],
       [6, 8]])

In [4]:
# 点积
d = np.dot(a, b)
d

array([[ 6,  6],
       [14, 14]])

### 二. 神经网络实现

In [10]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [11]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 溢出对策，减去输入中最大的值
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

#### 节点的加权和

In [6]:
X = np.array([1.0, 0.5])
W1 = np.array([[0.1,0.3,0.5], [0.2,0.4,0.6]])
B1 = np.array([0.1, 0.2, 0.3])

print(W1.shape)
print(X.shape)
print(B1.shape)

(2, 3)
(2,)
(3,)


In [8]:
A1 = np.dot(X, W1) + B1
A1

array([0.3, 0.7, 1.1])

#### 激活函数

In [10]:
Z1 = sigmoid(A1)
print(A1)
print(Z1)

[0.3 0.7 1.1]
[0.57444252 0.66818777 0.75026011]


In [12]:
def identity_function(x):
    return x

#### 完整实现

In [11]:
def init_network():
    network={}
    network['W1'] = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
    network['b1'] = np.array([0.1,0.2,0.3])
    network['W2'] = np.array([[0.1,0.4],[0.2,0.5],[0.3,0.6]])
    network['b2'] = np.array([0.1,0.2])
    network['W3'] = np.array([[0.1,0.3],[0.2,0.4]])
    network['b3'] = np.array([0.1,0.2])
    
    return network

In [15]:
def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    
    return y

In [16]:
network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


### 三. 入门示例

In [1]:
from mnist import load_mnist  # 从mnist.py导入

#### 1. 读取数据

In [2]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

/Users/shenjiayun/Documents/workspace/python/jupyter/深度学习入门/mnist/mnist.pkl
/Users/shenjiayun/Documents/workspace/python/jupyter/深度学习入门/mnist/train-images-idx3-ubyte.gz
Done
/Users/shenjiayun/Documents/workspace/python/jupyter/深度学习入门/mnist/train-labels-idx1-ubyte.gz
Done
/Users/shenjiayun/Documents/workspace/python/jupyter/深度学习入门/mnist/t10k-images-idx3-ubyte.gz
Done
/Users/shenjiayun/Documents/workspace/python/jupyter/深度学习入门/mnist/t10k-labels-idx1-ubyte.gz
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [3]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


#### 2. 展示图片

In [4]:
import numpy as np
from PIL import Image

In [5]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [6]:
img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28,28) # 把图像的形状变成原来的尺寸，因为读取时将其转成一维数组
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


#### 3. 神经网络的推理

In [16]:
import pickle

In [7]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test

In [14]:
def init_network():
    # 加载已经训练过的模型
    with open("./mnist/sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    return network

In [9]:
def predict(network, x):
    W1, W2, W3 = network['W1'],network['W2'],network['W3']
    b1, b2, b3 = network['b1'],network['b2'],network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [17]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 获取概率最高的元素的索引
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt)/ len(x)))

/Users/shenjiayun/Documents/workspace/python/jupyter/深度学习入门/mnist/mnist.pkl
Accuracy:0.9352


In [18]:
# 批量处理
x, t = get_data()
network = init_network()

batch_size = 100 # 批数量
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
print("Accuracy:" + str(float(accuracy_cnt)/ len(x)))

/Users/shenjiayun/Documents/workspace/python/jupyter/深度学习入门/mnist/mnist.pkl
Accuracy:0.9352
